### Unzipping the files

In [ ]:
!apt install unzip
!unzip -q "/content/master_data.zip"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
replace master_data/testing/buildings/20057.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


## Importing tensorflow and Keras Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

## Building our model

In [ ]:
model1=tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(150,150,3)),
    tf.keras.layers.Conv2D(16,(3,3),activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(3,3),
    
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
   
    tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
    tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Conv2D(256,(3,3),activation='relu',padding='same'),
    tf.keras.layers.Conv2D(256,(3,3),activation='relu',padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.4),
    
  
    tf.keras.layers.Flatten(),
       
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(32,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(16,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    
    tf.keras.layers.Dense(6,activation='softmax')
])



In [ ]:
model1.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 148, 148, 16)      448       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 146, 146, 16)      2320      
_________________________________________________________________
batch_normalization_10 (Batc (None, 146, 146, 16)      64        
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 48, 48, 16)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 46, 46, 32)        4640      
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 44, 44, 32)        9248      
_________________________________________________________________
batch_normalization_11 (Batc (None, 44, 44, 32)       

## Distinguishing Training and Testing Data

In [ ]:
train_dir = '/content/master_data/training'
test_dir = '/content/master_data/testing/'

train_data_gen = ImageDataGenerator(rescale = 1.0/255)
train_genrator = train_data_gen.flow_from_directory(
    train_dir,
    target_size = (100,100),
    class_mode = 'categorical',
    batch_size = 64
)

test_data_gen = ImageDataGenerator(rescale = 1.0/255)
test_genrator = test_data_gen.flow_from_directory(
    test_dir,
    target_size = (100,100),
    class_mode = 'categorical',
    batch_size = 64
)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


## Fitting the Model

In [ ]:
model.fit(train_genrator,
                   epochs =10,
                   verbose = 1,
                   validation_data = test_genrator,
                   )

Epoch 1/10
220/220 [==============================] - 125s 568ms/step - loss: 0.1794 - acc: 0.9572 - val_loss: 5.3530 - val_acc: 0.5870
Epoch 2/10
220/220 [==============================] - 126s 570ms/step - loss: 0.1889 - acc: 0.9523 - val_loss: 5.6893 - val_acc: 0.6020
Epoch 3/10
220/220 [==============================] - 126s 571ms/step - loss: 0.1276 - acc: 0.9667 - val_loss: 6.2657 - val_acc: 0.6047
Epoch 4/10
220/220 [==============================] - 125s 568ms/step - loss: 0.2224 - acc: 0.9463 - val_loss: 5.8389 - val_acc: 0.6000
Epoch 5/10
220/220 [==============================] - 125s 570ms/step - loss: 0.2343 - acc: 0.9493 - val_loss: 6.1722 - val_acc: 0.6220
Epoch 6/10
220/220 [==============================] - 124s 565ms/step - loss: 0.1831 - acc: 0.9584 - val_loss: 6.8033 - val_acc: 0.5980
Epoch 7/10
220/220 [==============================] - 124s 565ms/step - loss: 0.1337 - acc: 0.9686 - val_loss: 6.4487 - val_acc: 0.6080
Epoch 8/10
220/220 [============================

## Predicting the labels for images

In [ ]:
from keras.preprocessing import image
import numpy as np
import imageio
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
from os import listdir

labels={0:"buildings",
    1:"forest",
    2:"glacier",
    3:"mountain",
    4:"sea",
    5:"street"}
predictions = []
for file in listdir("/content/seg_pred/seg_pred"):
    img = image.load_img('/content/seg_pred/seg_pred/'+file, target_size=(100, 100))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x/255.0
    pre = model.predict(x)
    p_test=np.argmax(pre)

    print("Predicted:",labels[p_test])
    predictions.append(labels[p_test])
predictions = np.array(predictions)

Streaming output truncated to the last 5000 lines.
Predicted: forest
Predicted: buildings
Predicted: mountain
Predicted: glacier
Predicted: mountain
Predicted: buildings
Predicted: buildings
Predicted: mountain
Predicted: mountain
Predicted: buildings
Predicted: glacier
Predicted: mountain
Predicted: sea
Predicted: glacier
Predicted: street
Predicted: sea
Predicted: mountain
Predicted: mountain
Predicted: buildings
Predicted: street
Predicted: glacier
Predicted: forest
Predicted: forest
Predicted: street
Predicted: sea
Predicted: forest
Predicted: street
Predicted: mountain
Predicted: sea
Predicted: street
Predicted: street
Predicted: buildings
Predicted: mountain
Predicted: street
Predicted: street
Predicted: mountain
Predicted: street
Predicted: forest
Predicted: sea
Predicted: buildings
Predicted: street
Predicted: mountain
Predicted: sea
Predicted: street
Predicted: street
Predicted: glacier
Predicted: buildings
Predicted: sea
Predicted: buildings
Predicted: sea
Predicted: sea
Pred

## Making a CSV file of the predicted labels

In [ ]:
import numpy as np
import pandas as pd
prediction = pd.DataFrame(predictions, columns=['predictions']).to_csv('predictions.csv')